# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [88]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
from datetime import timedelta
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [89]:
# Format today and yesterday into strings for filename creation
format = "%m-%d-%Y"
now = datetime.now()
today = now.strftime(format)
yesterday = (now - timedelta(days=1)).strftime(format)

In [90]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    jh_live_date = today
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    jh_live_date = yesterday
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
print(jh_live_date)

08-31-2022


In [91]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2022-09-01 04:21:03,33.93911,67.709953,193250,7777,NaN,NaN,Afghanistan,496.424957,4.024321
1,NaN,NaN,NaN,Albania,2022-09-01 04:21:03,41.15330,20.168300,329352,3582,NaN,NaN,Albania,11444.575718,1.087590
2,NaN,NaN,NaN,Algeria,2022-09-01 04:21:03,28.03390,1.659600,270304,6878,NaN,NaN,Algeria,616.414073,2.544542
3,NaN,NaN,NaN,Andorra,2022-09-01 04:21:03,42.50630,1.521800,46027,154,NaN,NaN,Andorra,59570.309972,0.334586
4,NaN,NaN,NaN,Angola,2022-09-01 04:21:03,-11.20270,17.873900,102636,1917,NaN,NaN,Angola,312.283707,1.867766


Now immediately export the original jh dataset before making any changes

In [92]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [93]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/22/22,8/23/22,8/24/22,8/25/22,8/26/22,8/27/22,8/28/22,8/29/22,8/30/22,8/31/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,191040,191247,191585,191967,191967,191967,192463,192906,193004,193250
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,326181,326787,327232,327607,327961,328299,328515,328571,329017,329352
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,269894,269971,270043,270097,270145,270175,270194,270235,270272,270304


In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [95]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/22/22,8/23/22,8/24/22,8/25/22,8/26/22,8/27/22,8/28/22,8/29/22,8/30/22,8/31/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7767,7768,7769,7771,7771,7771,7777,7777,7777,7777
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3576,3577,3578,3579,3580,3581,3581,3581,3582,3582
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6878,6878,6878,6878,6878,6878,6878,6878,6878,6878


In [96]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [97]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/22/22,8/23/22,8/24/22,8/25/22,8/26/22,8/27/22,8/28/22,8/29/22,8/30/22,8/31/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [98]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [99]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/22/22,8/23/22,8/24/22,8/25/22,8/26/22,8/27/22,8/28/22,8/29/22,8/30/22,8/31/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,17864,17864,17864,17991,17991,17991,17991,17991,17991,17991
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,63514,63514,63514,64095,64095,64095,64095,64095,64095,64095
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,6553,6553,6553,6650,6650,6650,6650,6650,6650,6650


In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [101]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/22/22,8/23/22,8/24/22,8/25/22,8/26/22,8/27/22,8/28/22,8/29/22,8/30/22,8/31/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,222,222,222,222,222,222,222,222,222,222
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,693,693,693,693,693,693,693,693,693,693
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,101,101,101,101,101,101,101,101,101,101


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [103]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
675,1001.0,Autauga,Alabama,US,2022-09-01 04:21:03,32.539527,-86.644082,17991,222,NaN,NaN,"Autauga, Alabama, US",32202.115663,1.233950
712,1075.0,Lamar,Alabama,US,2022-09-01 04:21:03,33.779950,-88.096680,4249,66,NaN,NaN,"Lamar, Alabama, US",30778.703368,1.553307
713,1077.0,Lauderdale,Alabama,US,2022-09-01 04:21:03,34.901719,-87.656247,27464,404,NaN,NaN,"Lauderdale, Alabama, US",29617.487517,1.471017


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [104]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [105]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,94532122,1046243,0.0
India,44436339,527911,0.0
France,34745833,155133,0.0


In [106]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [107]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [108]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,94532122,1046243,0.0,93485879.0,2022-09-01 04:21:03,37.934269,-91.444083
India,44436339,527911,0.0,43908428.0,2022-09-01 04:21:03,23.088275,81.806127
France,34745833,155133,0.0,34590700.0,2022-09-01 04:21:03,6.430808,-34.730285
Brazil,34429853,683965,0.0,33745888.0,2022-09-01 04:21:03,-12.669522,-48.480493
Germany,32184553,147494,0.0,32037059.0,2022-09-01 04:21:03,51.571844,10.277881


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [109]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [110]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,11090871,94885,0.0
Texas,7848986,90299,0.0
Florida,7042096,79853,0.0


In [111]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [112]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [113]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,11090871,94885,0.0,10995986.0,2022-09-01 04:21:03,37.843962,-120.728594
Texas,7848986,90299,0.0,7758687.0,2022-09-01 04:21:03,31.660643,-98.653069
Florida,7042096,79853,0.0,6962243.0,2022-09-01 04:21:03,28.940755,-82.700744
New York,5988384,71124,0.0,5917260.0,2022-09-01 04:21:03,42.544151,-75.474183
Illinois,3687549,39328,0.0,3648221.0,2022-09-01 04:21:03,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [114]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [115]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,3408413,33155,0.0
Florida,Miami-Dade,1438468,11598,0.0
Arizona,Maricopa,1415836,17792,0.0
Illinois,Cook,1379109,14717,0.0
Texas,Harris,1195651,11238,0.0


In [116]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [117]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [118]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,3408413,33155,0.0,3375258.0,2022-09-01 04:21:03,34.308284,-118.228241,6037.0
Florida,Miami-Dade,1438468,11598,0.0,1426870.0,2022-09-01 04:21:03,25.611236,-80.551706,12086.0
Arizona,Maricopa,1415836,17792,0.0,1398044.0,2022-09-01 04:21:03,33.348359,-112.491815,4013.0
Illinois,Cook,1379109,14717,0.0,1364392.0,2022-09-01 04:21:03,41.841448,-87.816588,17031.0
Texas,Harris,1195651,11238,0.0,1184413.0,2022-09-01 04:21:03,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [119]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [120]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [121]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-08-29,192906,328571,270235,46027,102636,11,8949,9678225,434398,10018037,...,979160,243743,11793,542281,11405711,698384,535,11925,332822,256704
2022-08-30,193004,329017,270272,46027,102636,11,8974,9678225,434398,10030575,...,979160,243784,11793,542281,11408952,701739,535,11926,332822,256708
2022-08-31,193250,329352,270304,46027,102636,11,8974,9678225,434398,10042081,...,979160,243801,11815,542508,11411679,702332,535,11926,332822,256726


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [122]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [123]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-08-29,7777,3581,6878,154,1917,0,145,129711,8655,13834,...,7445,1637,14,5796,43113,5694,0,2155,4016,5593
2022-08-30,7777,3582,6878,154,1917,0,145,129711,8655,13900,...,7445,1637,14,5796,43117,5700,0,2155,4016,5593
2022-08-31,7777,3582,6878,154,1917,0,145,129711,8655,13961,...,7445,1637,14,5796,43117,5704,0,2155,4016,5596


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [124]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [125]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-08-29,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-08-30,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-08-31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [126]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/22/22,8/23/22,8/24/22,8/25/22,8/26/22,8/27/22,8/28/22,8/29/22,8/30/22,8/31/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,17864,17864,17864,17991,17991,17991,17991,17991,17991,17991
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,4201,4201,4201,4249,4249,4249,4249,4249,4249,4249
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,27045,27045,27045,27464,27464,27464,27464,27464,27464,27464
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,8448,8448,8448,8590,8590,8590,8590,8590,8590,8590
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,42678,42678,42678,43115,43115,43115,43115,43115,43115,43115


In [127]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [128]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [129]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-08-29,1479605,293076,8168,2245713,930076,11063718,1628992,873420,302387,49,...,2292790,7829851,1026812,139769,22622,2034647,1782970,580697,1829224,174213
2022-08-30,1479605,293076,8168,2245713,931459,11087642,1629832,874415,302522,49,...,2292790,7841112,1026812,139769,22669,2037406,1782970,581638,1830621,174828
2022-08-31,1479605,294498,8168,2254374,932639,11090871,1631016,875076,302738,49,...,2296285,7848986,1026812,140308,22711,2040106,1784728,582666,1832391,174828


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [130]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [131]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2022-08-29       17991   64095    6650  7291  16496    2715   5843   35737   
2022-08-30       17991   64095    6650  7291  16496    2715   5843   35737   
2022-08-31       17991   64095    6650  7291  16496    2715   5843   35737   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-08-29         9800     5805  ...    7337   2121     9117     2159   
2022-08-30         9800     5805  ...    7358   2127     9150     2160   
2022-08-31         9800     5805  ...    7358   2127     9150     2160   

Province_State                                                     
Admin2         Sweetwater  Teton Uinta Unassigned Washakie Weston  
2022-08-29          12024  11483  6045          0     2616   1783  
2022-08-30          12063  11526  6061          0     2631   1790  
2022-08-31          12063  11526  6061          0     2631   1790  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [132]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [133]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-08-29,20048,1338,34,31047,11905,94840,13156,11197,3042,0,...,27427,90224,4968,707,122,21374,14010,7274,15059,1877
2022-08-30,20048,1338,34,31047,11910,94864,13162,11259,3042,0,...,27427,90268,4968,707,122,21395,14010,7277,15063,1881
2022-08-31,20048,1338,34,31114,11918,94885,13166,11269,3042,0,...,27442,90299,4968,707,122,21416,14039,7286,15072,1881


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [134]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [135]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-08-29         222     693     101  105    251      54    129     640   
2022-08-30         222     693     101  105    251      54    129     640   
2022-08-31         222     693     101  105    251      54    129     640   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-08-29          165       87  ...     149     42       81       27   
2022-08-30          165       87  ...     149     43       81       27   
2022-08-31          165       87  ...     149     43       81       27   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-08-29            129    16    40          0       47     20  
2022-08-30            130    16    40          0       47     20  
2022-08-31            130    16    40          0       47     20  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [136]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [137]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [138]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [139]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [140]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [141]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [142]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [143]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [144]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [145]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [146]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change().round(7)
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-08-29,0.002302,0.000170,0.000152,0.0,0.0,0.0,0.000000,0.0,0.004914,0.001217,...,0.001898,0.000000,0.000764,0.000053,0.000211,0.000000,0.0,0.000000,0.000337,0.000019
2022-08-30,0.000508,0.001357,0.000137,0.0,0.0,0.0,0.002794,0.0,0.000000,0.001252,...,0.000000,0.000168,0.000000,0.000000,0.000284,0.004804,0.0,0.000084,0.000000,0.000016
2022-08-31,0.001275,0.001018,0.000118,0.0,0.0,0.0,0.000000,0.0,0.000000,0.001147,...,0.000000,0.000070,0.001865,0.000419,0.000239,0.000845,0.0,0.000000,0.000000,0.000070


In [147]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [148]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change().round(7)
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-08-29,0.0,0.000000,0.0,0.0,0.0,NaN,0.0,0.0,0.000809,0.005378,...,0.000941,0.0,0.0,0.0,0.000023,0.000000,NaN,0.0,0.0,0.000000
2022-08-30,0.0,0.000279,0.0,0.0,0.0,NaN,0.0,0.0,0.000000,0.004771,...,0.000000,0.0,0.0,0.0,0.000093,0.001054,NaN,0.0,0.0,0.000000
2022-08-31,0.0,0.000000,0.0,0.0,0.0,NaN,0.0,0.0,0.000000,0.004389,...,0.000000,0.0,0.0,0.0,0.000000,0.000702,NaN,0.0,0.0,0.000536


In [149]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [150]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change().round(7)
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-08-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-08-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [151]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [152]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change().round(7)
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-08-29,0.0,0.000000,0.0,0.000000,0.000503,0.000578,0.001221,0.000144,0.001955,0.0,...,0.000000,0.002257,0.0,0.000000,0.002704,0.003041,0.002558,0.003737,0.001721,0.00000
2022-08-30,0.0,0.000000,0.0,0.000000,0.001487,0.002162,0.000516,0.001139,0.000446,0.0,...,0.000000,0.001438,0.0,0.000000,0.002078,0.001356,0.000000,0.001620,0.000764,0.00353
2022-08-31,0.0,0.004852,0.0,0.003857,0.001267,0.000291,0.000727,0.000756,0.000714,0.0,...,0.001524,0.001004,0.0,0.003856,0.001853,0.001325,0.000986,0.001767,0.000967,0.00000


In [153]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [154]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change().round(7)
jh_hist_uscounties_cases_pct.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-08-29         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-08-30         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-08-31         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ...   Wyoming                               \
Admin2         Chambers Cherokee  ...      Park    Platte Sheridan  Sublette   
2022-08-29          0.0      0.0  ...  0.000000  0.000000  0.00000  0.000000   
2022-08-30          0.0      0.0  ...  0.002862  0.002829  0.00362  0.000463   
2022-08-31          0.0      0.0  ...  0.000000  0.000000  0.00000  0.000000   

Province_State                                                                
Admin2         Sweetwater     Teton     Uinta Unassigned  Washakie    Weston  
2022-08-29       0.000000  0.000000  0.000000        NaN  0.000000  0.000000  
2022-08-30       0.003243  0.003745  0.002647        NaN  0.005734  0.003926  
2022-08-31       0.000000  0.000000  0.000000        NaN  0.000000  0.000000  

[3 rows x 3336 columns]

In [155]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [156]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change().round(7)
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-08-29,0.0,0.0,0.0,0.000000,0.000000,0.000295,0.000380,0.000000,0.0,NaN,...,0.000000,0.000466,0.0,0.0,0.0,0.000843,0.000357,0.000825,0.001130,0.000000
2022-08-30,0.0,0.0,0.0,0.000000,0.000420,0.000253,0.000456,0.005537,0.0,NaN,...,0.000000,0.000488,0.0,0.0,0.0,0.000983,0.000000,0.000412,0.000266,0.002131
2022-08-31,0.0,0.0,0.0,0.002158,0.000672,0.000221,0.000304,0.000888,0.0,NaN,...,0.000547,0.000343,0.0,0.0,0.0,0.000981,0.002070,0.001237,0.000598,0.000000


In [157]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [158]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change().round(7)
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-08-29         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-08-30         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-08-31         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ... Wyoming                             \
Admin2         Chambers Cherokee  ...    Park   Platte Sheridan Sublette   
2022-08-29          0.0      0.0  ...     0.0  0.00000      0.0      0.0   
2022-08-30          0.0      0.0  ...     0.0  0.02381      0.0      0.0   
2022-08-31          0.0      0.0  ...     0.0  0.00000      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-08-29       0.000000   0.0   0.0        NaN      0.0    0.0  
2022-08-30       0.007752   0.0   0.0        NaN      0.0    0.0  
2022-08-31       0.000000   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [159]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [160]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean().round(7)
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-08-29,0.001908,0.000525,0.000133,0.000018,0.0,0.0,0.000096,0.000518,0.002474,0.001102,...,0.000957,0.000008,0.000429,0.000125,0.000202,0.000005,0.0,0.000044,0.000183,0.000034
2022-08-30,0.001208,0.000941,0.000135,0.000009,0.0,0.0,0.001445,0.000259,0.001237,0.001177,...,0.000479,0.000088,0.000215,0.000062,0.000243,0.002405,0.0,0.000064,0.000092,0.000025
2022-08-31,0.001241,0.000980,0.000127,0.000005,0.0,0.0,0.000722,0.000130,0.000619,0.001162,...,0.000239,0.000079,0.001040,0.000240,0.000241,0.001625,0.0,0.000032,0.000046,0.000047


In [161]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [162]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-08-29,0.000207,0.000068,0.0,0.0,0.0,NaN,0.000108,0.000127,0.000407,0.003776,...,0.000475,0.0,0.0,0.000121,0.000023,0.000002,NaN,0.000008,0.0,0.000006
2022-08-30,0.000104,0.000174,0.0,0.0,0.0,NaN,0.000054,0.000063,0.000204,0.004273,...,0.000238,0.0,0.0,0.000061,0.000058,0.000528,NaN,0.000004,0.0,0.000003
2022-08-31,0.000052,0.000087,0.0,0.0,0.0,NaN,0.000027,0.000032,0.000102,0.004331,...,0.000119,0.0,0.0,0.000030,0.000029,0.000615,NaN,0.000002,0.0,0.000270


In [163]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [164]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean().round(7)
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-08-29,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-08-30,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-08-31,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078


In [165]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [166]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean().round(7)
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-08-29,0.000337,0.000119,0.000490,0.000060,0.000651,0.000503,0.000694,0.000300,0.001076,0.0,...,0.000292,0.001278,0.000097,0.000061,0.001669,0.001698,0.001381,0.002090,0.000989,0.000033
2022-08-30,0.000169,0.000059,0.000245,0.000030,0.001069,0.001333,0.000605,0.000719,0.000761,0.0,...,0.000146,0.001358,0.000048,0.000031,0.001873,0.001527,0.000691,0.001855,0.000876,0.001782
2022-08-31,0.000084,0.002456,0.000122,0.001943,0.001168,0.000812,0.000666,0.000738,0.000738,0.0,...,0.000835,0.001181,0.000024,0.001943,0.001863,0.001426,0.000838,0.001811,0.000922,0.000891


In [167]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [168]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean().round(7)
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2022-08-29      0.000224  0.000288  0.000466  0.000548  0.000253  0.000410   
2022-08-30      0.000112  0.000144  0.000233  0.000274  0.000126  0.000205   
2022-08-31      0.000056  0.000072  0.000117  0.000137  0.000063  0.000102   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2022-08-29      0.000394  0.000277  0.000305  0.000374  ...  0.000019   
2022-08-30      0.000197  0.000139  0.000153  0.000187  ...  0.001441   
2022-08-31      0.000098  0.000069  0.000076  0.000093  ...  0.000720   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2022-08-29      0.000026  0.000038  0.000033   0.000028  0.000030  0.000012   
2022-08-30      0.001427  0.001829  0.000248   0.001636  0.001888  0.001329   
2022-08-31      0.000714  0.000914  0.000124   0.000818  0.000944  0.000665   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2022-08-29      -0.503937  0.000055  0.000080  
2022-08-30      -0.503937  0.002894  0.002003  
2022-08-31      -0.503937  0.001447  0.001002  

[3 rows x 3336 columns]

In [169]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [170]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-08-29,0.000011,0.000094,0.000947,0.000033,0.000190,0.000282,0.000253,0.000022,0.000031,NaN,...,0.000074,0.000287,0.000095,0.000112,0.0,0.000549,0.000353,0.000511,0.000626,0.000025
2022-08-30,0.000006,0.000047,0.000474,0.000017,0.000305,0.000268,0.000355,0.002780,0.000016,NaN,...,0.000037,0.000387,0.000048,0.000056,0.0,0.000766,0.000177,0.000462,0.000446,0.001078
2022-08-31,0.000003,0.000023,0.000237,0.001087,0.000488,0.000245,0.000329,0.001834,0.000008,NaN,...,0.000292,0.000365,0.000024,0.000028,0.0,0.000874,0.001123,0.000849,0.000522,0.000539


In [171]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [172]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-08-29         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-08-30         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-08-31         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ...       Wyoming                     \
Admin2         Chambers Cherokee  ...          Park    Platte Sheridan   
2022-08-29          0.0      0.0  ...  4.000000e-07  0.000000     -0.0   
2022-08-30          0.0      0.0  ...  2.000000e-07  0.011905     -0.0   
2022-08-31          0.0      0.0  ...  1.000000e-07  0.005952     -0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-08-29          0.0   0.000000   0.0   0.0       -1.0      0.0    0.0  
2022-08-30          0.0   0.003876   0.0   0.0       -1.0      0.0    0.0  
2022-08-31          0.0   0.001938   0.0   0.0       -1.0      0.0    0.0  

[3 rows x 3336 columns]

In [173]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [174]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

HTTPError: HTTP Error 403: Forbidden

In [ ]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [ ]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

In [ ]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [ ]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

In [ ]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

#### 